In [1]:
!wget https://www.dropbox.com/s/jvy39twpx6ekmij/596958_1073629_bundle_archive.zip?dl=0

--2020-08-17 21:02:50--  https://www.dropbox.com/s/jvy39twpx6ekmij/596958_1073629_bundle_archive.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.82.1, 2620:100:6032:1::a27d:5201
Connecting to www.dropbox.com (www.dropbox.com)|162.125.82.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/jvy39twpx6ekmij/596958_1073629_bundle_archive.zip [following]
--2020-08-17 21:02:50--  https://www.dropbox.com/s/raw/jvy39twpx6ekmij/596958_1073629_bundle_archive.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9f1d218ab21e58c69956b05746.dl.dropboxusercontent.com/cd/0/inline/A9qfra591kr1HgTpeMRq1OA8FhZTPjgkTBRcXGxnypD-Pl0RFCPAiqkKbwrvdgr9Jx2KYh3ksYsogU1gJQCqjGs8-D2ee4QLtx7qxtgoau8acfxcfREXpAJLAm23hxvMcP8/file# [following]
--2020-08-17 21:02:50--  https://uc9f1d218ab21e58c69956b05746.dl.dropboxusercontent.com/cd/0/inline/A9qfra591kr1HgTpeMRq1OA8FhZTPjgkTBRcXGxnypD

In [2]:
!unzip '596958_1073629_bundle_archive.zip?dl=0' -d "data/"

Archive:  596958_1073629_bundle_archive.zip?dl=0
  inflating: data/Placement_Data_Full_Class.csv  


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df=pd.read_csv('/content/data/Placement_Data_Full_Class.csv')
print(df.head())

   sl_no gender  ssc_p    ssc_b  ...  specialisation  mba_p      status    salary
0      1      M  67.00   Others  ...          Mkt&HR  58.80      Placed  270000.0
1      2      M  79.33  Central  ...         Mkt&Fin  66.28      Placed  200000.0
2      3      M  65.00  Central  ...         Mkt&Fin  57.80      Placed  250000.0
3      4      M  56.00  Central  ...          Mkt&HR  59.43  Not Placed       NaN
4      5      M  85.80  Central  ...         Mkt&Fin  55.50      Placed  425000.0

[5 rows x 15 columns]


In [11]:
input_cols=['gender','ssc_p','ssc_b','hsc_p','hsc_b','hsc_s','degree_p','degree_t','workex','etest_p','specialisation','mba_p','salary']
output_cols=['status']
X=df[input_cols]
Y=df[output_cols]

In [12]:
cols=df.columns
print(cols)

Index(['sl_no', 'gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s',
       'degree_p', 'degree_t', 'workex', 'etest_p', 'specialisation', 'mba_p',
       'status', 'salary'],
      dtype='object')


In [13]:
X['salary']=X['salary'].replace(np.nan,0)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
# lets do some data preprocessing :
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
Y=le.fit_transform(Y)
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()
X=encoder.fit_transform(X)
#print(X.head())

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
#for col in input_cols:
#  X[col]=le.fit_transform(X[col])

In [16]:
# lets split our data :
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3)
print(X_train.shape,Y_train.shape,X_test.shape,Y_test.shape)

(150, 656) (150,) (65, 656) (65,)


In [17]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(X_train,Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
lr.score(X_test,Y_test)

1.0

In [19]:
# lets check on RandomForestClassifier :
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=50,max_depth=5,random_state=2,criterion='gini')

In [20]:
rf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=2, verbose=0,
                       warm_start=False)

In [21]:
rf.score(X_test,Y_test)

0.6923076923076923

In [22]:
from sklearn.model_selection import cross_val_score

In [23]:
acc=[]
for i in range(1,50):
  accuracy=cross_val_score(RandomForestClassifier(n_estimators=i,criterion='entropy',max_depth=5,random_state=5),X_train,Y_train,cv=5).mean()
  acc.append(accuracy)

In [24]:
# 6 estimators is optimal and we are getting highest acuracy
print(np.argmax(acc))

13


In [25]:
rf=RandomForestClassifier(n_estimators=13,criterion='entropy',max_depth=5,random_state=10)
rf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=13,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [26]:
rf.score(X_test,Y_test)

0.7692307692307693

In [27]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(criterion='gini',max_depth=10,random_state=4)

In [28]:
dt.fit(X_train,Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=4, splitter='best')

In [29]:
dt.score(X_test,Y_test)

1.0

In [30]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB

In [31]:
gnb=GaussianNB()

In [33]:
gnb.fit(X_train.toarray(),Y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [34]:
gnb.score(X_test.toarray(),Y_test)

0.9230769230769231

In [35]:
mnb=MultinomialNB()
mnb.fit(X_train,Y_train)
mnb.score(X_test,Y_test)

0.8923076923076924

In [37]:
from sklearn.neighbors import KNeighborsClassifier
knb=KNeighborsClassifier(n_neighbors=25)
knb.fit(X_train,Y_train)
knb.score(X_test,Y_test)

0.8307692307692308

In [38]:
acc_list=[]
for i in range(1,50):
  acc=KNeighborsClassifier(n_neighbors=i).fit(X_train,Y_train).score(X_test,Y_test)
  acc_list.append(acc)

In [39]:
print(np.argmax(acc_list))

5


In [40]:
print(acc_list)

[0.7846153846153846, 0.8153846153846154, 0.8307692307692308, 0.8615384615384616, 0.9076923076923077, 0.9384615384615385, 0.8923076923076924, 0.8923076923076924, 0.9076923076923077, 0.8923076923076924, 0.9230769230769231, 0.9076923076923077, 0.8769230769230769, 0.8923076923076924, 0.9230769230769231, 0.9230769230769231, 0.9230769230769231, 0.9076923076923077, 0.8769230769230769, 0.9076923076923077, 0.8769230769230769, 0.8615384615384616, 0.8615384615384616, 0.8461538461538461, 0.8307692307692308, 0.8923076923076924, 0.8461538461538461, 0.8923076923076924, 0.8307692307692308, 0.8461538461538461, 0.8153846153846154, 0.8461538461538461, 0.8307692307692308, 0.8461538461538461, 0.8, 0.8461538461538461, 0.8153846153846154, 0.8153846153846154, 0.8153846153846154, 0.8307692307692308, 0.8153846153846154, 0.8153846153846154, 0.8, 0.8, 0.7846153846153846, 0.8, 0.7538461538461538, 0.7846153846153846, 0.7384615384615385]


In [41]:
knb=KNeighborsClassifier(n_neighbors=6)
knb.fit(X_train,Y_train)
knb.score(X_test,Y_test)

0.9384615384615385

In [42]:
# Although logistic rgression is giving highest accuracy of 100% which is ofcourse a case of overfitting, so K nearest neighbouring is giving highest accuracy of 93% after that 